In [11]:
# i want to finetune a meta-llama/Llama-3.2-1B model to do credit card fraud detection. Log everything to wandb. 
# the huggingface dataset for legitimate transactions is called "yunfan-y/fraud-detection-legitimate"
# the huggingface dataset for fraudulent transactions is called "yunfan-y/fraud-detection-fraud"
# all dataset has been split into train, validation and test set. 
# all datasets have columns "conversation" and "response"
# the response is either "LEGITIMATE" or "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: fraud_Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR
# response: LEGITIMATE

# randomly select half of the training set to shorten the training time

# after the model is trained, i want to evaluate the model on the test set. 

In [12]:
import random
import wandb
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

wandb.login()
wandb.init(project="fraud_detection_llama")

# Load legitimate transactions dataset
legit_datasets = load_dataset("yunfan-y/fraud-detection-legitimate")

# Load fraudulent transactions dataset
fraud_datasets = load_dataset("yunfan-y/fraud-detection-fraud")

# Combine the training sets
train_dataset = concatenate_datasets([legit_datasets['train'], fraud_datasets['train']])

# Combine the validation sets
validation_dataset = concatenate_datasets([legit_datasets['validation'], fraud_datasets['validation']])

# Combine the test sets
test_dataset = concatenate_datasets([legit_datasets['test'], fraud_datasets['test']])

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [13]:
# Calculate half of the training dataset size
train_size = len(train_dataset)
half_train_size = train_size // 2

# Randomly select half of the indices
random_indices = random.sample(range(train_size), half_train_size)

# Select the subset of the training dataset
train_dataset = train_dataset.select(random_indices)

In [14]:
from transformers import AutoTokenizer, LlamaForSequenceClassification

# Update the model name to an existing one
model_name = "meta-llama/Llama-3.2-1B"

# Load the tokenizer using AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# Assign the pad_token as eos_token (if not already set)
tokenizer.pad_token = tokenizer.eos_token

# Load the model with a sequence classification head
model = LlamaForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    problem_type="single_label_classification",
)

# Set the pad_token_id in the model configuration
model.config.pad_token_id = tokenizer.pad_token_id

# Resize the token embeddings to match the tokenizer
model.resize_token_embeddings(len(tokenizer))

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(128256, 2048)

In [15]:
def preprocess_function(examples):
    return tokenizer(
        examples['conversation'],
        truncation=True,
        padding='max_length',  # Ensures all sequences are the same length
        max_length=512,
    )

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
validation_dataset = validation_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 43002/43002 [00:05<00:00, 8374.66 examples/s]


In [16]:
# Define a mapping from response text to numerical labels
label_mapping = {
    "LEGITIMATE": 0,
    "FRAUD": 1
}

# Function to map responses to labels
def encode_labels(example):
    example['label'] = label_mapping[example['response']]
    return example

# Apply the mapping to the datasets
train_dataset = train_dataset.map(encode_labels)
validation_dataset = validation_dataset.map(encode_labels)
test_dataset = test_dataset.map(encode_labels)

# Now rename 'label' to 'labels' and set the format for PyTorch
train_dataset = train_dataset.rename_column("label", "labels")
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

validation_dataset = validation_dataset.rename_column("label", "labels")
validation_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 43002/43002 [00:01<00:00, 27617.67 examples/s]


In [17]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    logging_steps=100,
    num_train_epochs=1,
    per_device_train_batch_size=2,  # Adjust batch size based on GPU memory
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="wandb",
)

c:\Users\Aiden\AppData\Local\anaconda3\envs\hugging_face\Lib\site-packages\transformers\training_args.py:1541: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

eval_results = trainer.evaluate(eval_dataset=test_dataset)
print(f"Test Evaluation results: {eval_results}")

model.push_to_hub("yunfan-y/fraud-detection-fine-tune-origin")

wandb.log(eval_results)
wandb.finish()

KeyboardInterrupt: 